In [18]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            bbx = member.find('bndbox')
            xmin = int(bbx.find('xmin').text)
            ymin = int(bbx.find('ymin').text)
            xmax = int(bbx.find('xmax').text)
            ymax = int(bbx.find('ymax').text)
            label = member.find('name').text

            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     label,
                     xmin,
                     ymin,
                     xmax,
                     ymax
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [19]:
datasets = 'train'
path = os.path.join(os.getcwd(), datasets , "*.xml")
all_xml = glob.glob(path)
len(all_xml)

3003

In [20]:
# 231.xml has error in parsing as it was not stored in VOC XML format
df = pd.DataFrame()
for i in all_xml:
    try:
        xml_df  = xml_to_csv(i)
        xml_df["image_name"] = i
        df = df.append(xml_df, ignore_index = True)
    except:
        continue

In [21]:
# EOL error if you put \ at the end of a string. Replacing it with \\ will work fine
left = "/home/aifahim/projects/Dhaka AI/train/"
df['image_name'] = df['image_name'].map(lambda x: x.lstrip(left).rstrip('.xml')) + ".jpg"

In [22]:
df.drop("filename",axis=1, inplace = True)

In [27]:
df.tail()

,width,height,class,xmin,ymin,xmax,ymax,image_name
24363,1058,624,three wheelers (CNG),590,228,679,341,_442.jpg
24364,1058,624,truck,816,178,878,248,_442.jpg
24365,1058,624,truck,966,185,1028,242,_442.jpg
24366,1058,624,truck,916,179,953,229,_442.jpg
24367,1058,624,car,889,202,907,220,_442.jpg


In [24]:
# checking
df["image_name"].nunique()

2965

In [26]:
# convert to csv 
df.to_csv("dhaka_ai.csv", index=False)